In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pima-indians-diabetes-database/diabetes.csv


# Read data

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv("/kaggle/input/pima-indians-diabetes-database/diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [5]:
df.shape

(768, 9)

# Extract features and target

In [6]:
# feature and target
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [7]:
X.shape

(768, 8)

In [8]:
y.shape

(768,)

# Data Preprocessing

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [10]:
X = scaler.fit_transform(X)

# Spliting dataset

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

# Build Model

In [12]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense 

In [13]:
model = Sequential()

model.add(Dense(32, activation='relu', input_dim=8))
model.add(Dense(1, activation='sigmoid'))

In [14]:
model.compile(optimizer='Adam',loss='binary_crossentropy', metrics= ['accuracy'])

In [15]:
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/100
20/20 [==============================] - 1s 15ms/step - loss: 0.7836 - accuracy: 0.4300 - val_loss: 0.7636 - val_accuracy: 0.4675
Epoch 2/100
20/20 [==============================] - 0s 3ms/step - loss: 0.7208 - accuracy: 0.5831 - val_loss: 0.7176 - val_accuracy: 0.5455
Epoch 3/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6760 - accuracy: 0.6482 - val_loss: 0.6839 - val_accuracy: 0.6104
Epoch 4/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6406 - accuracy: 0.6873 - val_loss: 0.6556 - val_accuracy: 0.6558
Epoch 5/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6114 - accuracy: 0.7052 - val_loss: 0.6311 - val_accuracy: 0.6753
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5879 - accuracy: 0.7020 - val_loss: 0.6102 - val_accuracy: 0.6818
Epoch 7/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5676 - accuracy: 0.7199 - val_loss: 0.5933 - val_accuracy: 0.6883
Epoch 8/100


# Keras Tuner for neurons

In [16]:
import keras_tuner as kt

In [17]:
def build_model(hp):
    model = Sequential()
    
    units = hp.Int('units', 8, 256, step=8)

    model.add(Dense(units=units, activation='relu', input_dim=8))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='Adam', loss='binary_crossentropy', metrics= ['accuracy'])
    
    return model

In [18]:
tuner = kt.RandomSearch(build_model, objective = 'val_accuracy', max_trials=5,
                       directory='my_dir')

In [19]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7857142686843872

Best val_accuracy So Far: 0.7857142686843872
Total elapsed time: 00h 00m 10s


In [20]:
tuner.get_best_hyperparameters()[0].values

{'units': 192}

In [21]:
model = tuner.get_best_models(num_models=1)[0]

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 192)               1728      
                                                                 
 dense_1 (Dense)             (None, 1)                 193       
                                                                 
Total params: 1,921
Trainable params: 1,921
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.fit(X_train, y_train, epochs=100, batch_size=32, initial_epoch=6, 
          validation_data=(X_test, y_test))

Epoch 7/100
20/20 [==============================] - 1s 12ms/step - loss: 0.4540 - accuracy: 0.7818 - val_loss: 0.4923 - val_accuracy: 0.7727
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4482 - accuracy: 0.7850 - val_loss: 0.4887 - val_accuracy: 0.7727
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4444 - accuracy: 0.7850 - val_loss: 0.4888 - val_accuracy: 0.7727
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4408 - accuracy: 0.7883 - val_loss: 0.4897 - val_accuracy: 0.7727
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4378 - accuracy: 0.7899 - val_loss: 0.4939 - val_accuracy: 0.7662
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4362 - accuracy: 0.7915 - val_loss: 0.4942 - val_accuracy: 0.7597
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4331 - accuracy: 0.7883 - val_loss: 0.4966 - val_accuracy: 0.7597
Epoch 14

# Keras tuner for no of layers

In [24]:
def build_model(hp):
    model = Sequential()

    model.add(Dense(240, activation='relu', input_dim=8))
    
    for i in range(hp.Int('num_layers', 1, 10)):
        model.add(Dense(240, activation='relu'))
        
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='Adam', loss='binary_crossentropy', metrics= ['accuracy'])
    
    return model

In [25]:
tuner = kt.RandomSearch(build_model, objective = 'val_accuracy', max_trials=5,
                       directory="my_dir2",
                       project_name='num_layers')

In [26]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 05s]
val_accuracy: 0.7402597665786743

Best val_accuracy So Far: 0.7727272510528564
Total elapsed time: 00h 00m 18s


In [27]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1}

In [28]:
model = tuner.get_best_models(num_models=1)[0]

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 240)               2160      
                                                                 
 dense_1 (Dense)             (None, 240)               57840     
                                                                 
 dense_2 (Dense)             (None, 1)                 241       
                                                                 
Total params: 60,241
Trainable params: 60,241
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.fit(X_train, y_train, epochs=100, batch_size=32, initial_epoch=6, 
          validation_data=(X_test, y_test))

Epoch 7/100
20/20 [==============================] - 1s 14ms/step - loss: 0.4481 - accuracy: 0.7850 - val_loss: 0.4956 - val_accuracy: 0.7597
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4303 - accuracy: 0.7980 - val_loss: 0.4959 - val_accuracy: 0.7403
Epoch 9/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4267 - accuracy: 0.8029 - val_loss: 0.4937 - val_accuracy: 0.7273
Epoch 10/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4157 - accuracy: 0.8046 - val_loss: 0.5022 - val_accuracy: 0.7403
Epoch 11/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4117 - accuracy: 0.7980 - val_loss: 0.5076 - val_accuracy: 0.7727
Epoch 12/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4021 - accuracy: 0.7980 - val_loss: 0.5040 - val_accuracy: 0.7338
Epoch 13/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4032 - accuracy: 0.8046 - val_loss: 0.5164 - val_accuracy: 0.7597
Epoch 14

# Keras tuner all in one

In [31]:
def build_model(hp):
    model = Sequential()
    
    counter = 0
    
    for i in range(hp.Int('num_layers', 1, 10)):
        
        if counter == 0:
            model.add(Dense(hp.Int('units', 8, 256, 8), activation=hp.Choice('activation'+str(i), values=['relu', 'tanh', 'sigmoid']),input_dim=8))
            
        else:
            model.add(Dense(hp.Int('units', 8, 256, 8), activation=hp.Choice('activation'+str(i), values=['relu', 'tanh', 'sigmoid']))) 
            
        counter+=1

    
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer=hp.Choice('optimizer',['Adam', 'sgd', 'rmsprop', 'adadelta']), loss='binary_crossentropy', metrics= ['accuracy'])
    
    return model

In [32]:
tuner = kt.RandomSearch(build_model, objective = 'val_accuracy', max_trials=3,
                       directory="my_dir4",
                       project_name='all')

In [33]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 3 Complete [00h 00m 03s]
val_accuracy: 0.7207792401313782

Best val_accuracy So Far: 0.7662337422370911
Total elapsed time: 00h 00m 07s


In [34]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 4,
 'units': 40,
 'activation0': 'tanh',
 'optimizer': 'rmsprop',
 'activation1': 'relu',
 'activation2': 'relu',
 'activation3': 'relu'}

In [35]:
model = tuner.get_best_models(num_models=1)[0]

In [36]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 40)                360       
                                                                 
 dense_1 (Dense)             (None, 40)                1640      
                                                                 
 dense_2 (Dense)             (None, 40)                1640      
                                                                 
 dense_3 (Dense)             (None, 40)                1640      
                                                                 
 dense_4 (Dense)             (None, 1)                 41        
                                                                 
Total params: 5,321
Trainable params: 5,321
Non-trainable params: 0
_________________________________________________________________


In [37]:
model.fit(X_train, y_train, epochs=200, batch_size=32, initial_epoch=6, 
          validation_data=(X_test, y_test))

Epoch 7/200
20/20 [==============================] - 1s 14ms/step - loss: 0.4780 - accuracy: 0.7818 - val_loss: 0.4907 - val_accuracy: 0.7662
Epoch 8/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4583 - accuracy: 0.7883 - val_loss: 0.4817 - val_accuracy: 0.7532
Epoch 9/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4503 - accuracy: 0.7964 - val_loss: 0.4862 - val_accuracy: 0.7597
Epoch 10/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4428 - accuracy: 0.8029 - val_loss: 0.4858 - val_accuracy: 0.7532
Epoch 11/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4364 - accuracy: 0.8013 - val_loss: 0.4957 - val_accuracy: 0.7532
Epoch 12/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4315 - accuracy: 0.7964 - val_loss: 0.4859 - val_accuracy: 0.7143
Epoch 13/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4257 - accuracy: 0.8062 - val_loss: 0.4893 - val_accuracy: 0.7532
Epoch 14

In [38]:
# dropout
# model.add(Dropout(hp.Choice('dropout', str(i), values=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])))